In [ ]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO)

In [ ]:
import pandas as pd

df = pd.read_parquet("output_dataset.parquet")
for col in df.columns:
    print(col)
df.head()

In [ ]:
from projects.pltv.feature_engineering import _timestamp_col_to_datetime, _convert_net_billings_to_avg_net_billings

_timestamp_col_to_datetime(df)
_convert_net_billings_to_avg_net_billings(df)
for col in df.columns:
    print(col)
df.head()


In [ ]:
MIN_COHORT_SIZE = 250

test_df = df.copy()
test_df.rename(
    columns={
        'START_DATE_MONTH': 'ds',
        'AVG_NET_BILLINGS_730_DAYS': 'y'
    },
    inplace=True # type: ignore
)


train_df = test_df[
    (test_df['y'].notna())
    & (test_df['GROSS_ADDS'] >= MIN_COHORT_SIZE)
].copy()

predict_df = test_df[
    (test_df['y'].isna())
    & (test_df['AVG_NET_BILLINGS_365_DAYS'].notna())
].copy()


train_df

In [ ]:
train_df["cap"] = train_df.y.astype(float).max() * 1.2
train_df["floor"] = 0

train_df.head()

# fill all numeric columns with 0
train_df.fillna(0, inplace=True)

train_df.head()


In [ ]:
from prophet import Prophet

m = Prophet(changepoint_prior_scale=0.82, growth="logistic")